In [48]:
import ee
import numpy as np
ee.Initialize()

In [251]:
image=ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_044034_19990723').select('B1','B2','B3','B4','B5','B6','B7')
scale = image.projection().nominalScale().getInfo()
print('scale',scale)
ndvi=ee.Image('LANDSAT/LE07/C01/T1/LE07_044034_19990707')
ndvi=ndvi.normalizedDifference(['B4', 'B3'])
mask=ndvi.lt(0.8)
masked_img=image.updateMask(mask)
#masked_img=masked_img.updateMask(masked_img)
maximg=masked_img.reduceRegion(reducer=ee.Reducer.max(),scale=30,bestEffort=True)
minimg=masked_img.reduceRegion(reducer=ee.Reducer.min(),scale=30,bestEffort=True)
print(masked_img.bandNames().getInfo())
#masked_img=masked_img.reduceRegion(reducer=ee.Reducer.toList(),bestEffort=True)
#print(masked_img)

for band in masked_img.bandNames().getInfo():
    hist=masked_img.select(band).reduceRegion(reducer=ee.Reducer.fixedHistogram(min=0,max=30000,steps=32),scale=30,bestEffort=True)
    hist_data=hist.get(band).getInfo()
    data=np.asarray(hist_data)
    print(data)
    if band == 'B1':
        hist3d=data[:,1]
        hist3d=np.expand_dims(hist3d,axis=1)
    else:
        hist3d=np.concatenate((hist3d,np.expand_dims(data[:,1],axis=1)),axis=1)
    print(hist3d.shape)



('scale', 30.0)
[u'B1', u'B2', u'B3', u'B4', u'B5', u'B6', u'B7']
[[  0.00000000e+00   3.59564594e+06]
 [  9.37500000e+02   7.53257345e+05]
 [  1.87500000e+03   4.34354569e+05]
 [  2.81250000e+03   3.31491176e+05]
 [  3.75000000e+03   0.00000000e+00]
 [  4.68750000e+03   0.00000000e+00]
 [  5.62500000e+03   1.34000000e+02]
 [  6.56250000e+03   7.32407490e+04]
 [  7.50000000e+03   8.99200000e+03]
 [  8.43750000e+03   1.08274510e+01]
 [  9.37500000e+03   0.00000000e+00]
 [  1.03125000e+04   3.91000000e+02]
 [  1.12500000e+04   1.03807533e+05]
 [  1.21875000e+04   0.00000000e+00]
 [  1.31250000e+04   0.00000000e+00]
 [  1.40625000e+04   1.34039216e+01]
 [  1.50000000e+04   1.02010514e+05]
 [  1.59375000e+04   0.00000000e+00]
 [  1.68750000e+04   0.00000000e+00]
 [  1.78125000e+04   0.00000000e+00]
 [  1.87500000e+04   0.00000000e+00]
 [  1.96875000e+04   4.56724166e+06]
 [  2.06250000e+04   0.00000000e+00]
 [  2.15625000e+04   0.00000000e+00]
 [  2.25000000e+04   0.00000000e+00]
 [  2.343

dateFilter = ee.Filter.date('2014-01-01', '2014-12-31')
modis_sr=ee.ImageCollection('MODIS/006/MOD09GA').select('sur_refl_b01,sur_refl_b02,sur_refl_b03,sur_refl_b04,sur_refl_b05,sur_refl_b06,sur_refl_b07').filter(dateFilter)
modis_lst=ee.ImageCollection('MODIS/006/MOD11A2').select('LST_Day_1km').filter(dateFilter)
modis_lc=ee.ImageCollection('MODIS/006/MCD12Q1').select('LC_Prop2').filter(dateFilter)
innerJoin = ee.Join.inner()
filterTimeEq = ee.Filter.equals(
  leftField= 'system:time_start',
  rightField= 'system:time_start'
)
simpleJoin = ee.Join.simple()
simpleJoinedMODIS = simpleJoin.apply(modis_lc, modis_lst, filterTimeEq)
print('Inner join output:', simpleJoinedMODIS.size().getInfo())

('Inner join output:', 46)
